In [4]:
import dask 
import numpy as np
import dask.dataframe as dd 
from collections import OrderedDict
#from sentence_transformers import SentenceTransformer
# from transformers import AutoTokenizer, AutoModel, TFAutoModel
# import torch
#from dask.dataframe import to_numeric

# Load data

In [ ]:
import os


os.environ["DASK_TEMPORARY_DIRECTORY"] = "smt/dask"
os.environ["PGEOCODE_DATA_DIR"] = "smt/pgeocode_data"
if not os.path.exists("smt"):
    os.mkdir("smt")
if not os.path.exists("smt/dask"):
    os.mkdir("smt/dask")
if not os.path.exists("smt/pgeocode_data"):
    os.mkdir("smt/pgeocode_data")
import pandas as pd
import hvplot.pandas
import dask
import hvplot.dask
import dask.dataframe as dd
import json
import tqdm
from scipy.spatial.distance import cdist
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
import pingouin as pg
import pgeocode as geo
import geopandas as gpd
import fiona
import panel as pn
pn.extension(comms="vscode")
from scipy.stats import entropy


### Specification of the data folder 
options: 
- "preflight" for a sample 
- "reduced" for all data

In [ ]:
DATA_FOLDER = "preflight"

### Load views data

In [ ]:
ddf_views = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER}/reduced_views.csv', dtype={
       'GEO_ZIPCODE': object,
       'REFR_MEDIUM': object, 'IS_LOGGED_IN': object, 'file_name': object,
       "privacy_advertising":int,       
       "privacy_analytics":int,       
       "privacy_functional":int,       
       "privacy_geo_location":int,       
       "privacy_marketing":int,       
       "privacy_non-personalised_ads":int,       
       "privacy_personalisation":int,       
       "privacy_social_media":int,       
       "privacy_target_advertising":int,       
       'ARTICLE_ID':"str",
       })
ddf_views["article_id"] = ddf_views["ARTICLE_ID"]
# ddf_views = ddf_views.drop(["ARTICLE_ID"] , axis=1)
ddf_views["DERIVED_TSTAMP"] = dd.to_datetime(ddf_views["DERIVED_TSTAMP"])
ddf_views["dt_hour"] = ddf_views["DERIVED_TSTAMP"].dt.hour
ddf_views["dt_weekday"] = ddf_views["DERIVED_TSTAMP"].dt.weekday
ddf_views["dt_dayofmonth"] = ddf_views["DERIVED_TSTAMP"].dt.day
ddf_views["dt_month"] = ddf_views["DERIVED_TSTAMP"].dt.month
ddf_views = ddf_views.replace("nu.web.advertorial", "nu.web")
ddf_views_unidentified = ddf_views[ddf_views["QUASI_USER_ID"].isna()].copy()
ddf_views = ddf_views[~ddf_views["QUASI_USER_ID"].isna()]
device_counts = ddf_views["APP_ID"].value_counts().compute()
# sample_amount = (device_counts.min()//100000)*100000
sample_amount = device_counts.min()
# sample_amount = 1000000
unique_devices = device_counts.index.unique()
columns_privacy = list(ddf_views.columns[ddf_views.columns.str.startswith("privacy_")].values)
columns_time = ["hour", "weekday", "dayofmonth", "month"]
ddf_views = ddf_views.groupby('APP_ID').apply(lambda df: df.sample(sample_amount if len(df) else 0) , meta=ddf_views.partitions[0]).reset_index(drop=True)
ddf_views = ddf_views.set_index('article_id')
ddf_views

In [ ]:
# grab a sample
ddf_sample, rest = ddf_views.random_split([0.0001, 0.9999])

In [ ]:
# select the columns you want to work with
views_small = ddf_sample.drop(["GEO_CITY", "GEO_REGION", "REFR_MEDIUM",'privacy_functional',
       'privacy_analytics', 'privacy_target_advertising',
       'privacy_personalisation', 'privacy_non-personalised_ads',
       'privacy_marketing', 'privacy_social_media', 'privacy_geo_location',
       'privacy_advertising', 'file_name'], axis = 1)

### Get some numbers 

In [ ]:
# How many non-NL GEO-COUNTRY? 

In [12]:
country_counts = ddf_views["GEO_COUNTRY"].value_counts().compute()
print(country_counts)
print(type(country_counts))

NL    12356429
ES      118474
DE      117140
FR      106793
BE       98041
        ...   
PM           1
VA           1
BL           1
FK           1
IO           1
Name: GEO_COUNTRY, Length: 223, dtype: int64
<class 'pandas.core.series.Series'>


In [15]:
country_counts.to_csv("country_counts_preflight.csv")

In [ ]:
# How many unique QUASI_USER_ID? 

In [6]:
id_counts = ddf_views["QUASI_USER_ID"].value_counts().compute()
print(id_counts)

dee1f36084ab7b8be4a9d993982f807a    753
b20d4f85f00f19ca790009bc1924856d    545
0322f1afbeb4505a71afce2d9364d87b    267
b8f0755f048b8bc50de36833db1b9103    257
2c928756a86f21b276fbd3a04971c289    235
                                   ... 
8d8bb4bf4b397e01548719507d569ce0      1
8d8bb71ea4efd573b706716c12b259a7      1
8d8bbf635aec85bf9574b5a27d943ad7      1
8d8bbfa5b98adbd2cd567d9ed7a5b586      1
800b9f7730ea538011b44d01aad64ad1      1
Name: QUASI_USER_ID, Length: 4316046, dtype: int64


In [ ]:
## Interaction 

drop = ['GEO_REGION', 'REFR_URLHOST', 'privacy_functional',
       'privacy_analytics', 'privacy_target_advertising',
       'privacy_personalisation', 'privacy_non-personalised_ads',
       'privacy_marketing', 'privacy_social_media', 'privacy_geo_location',
       'privacy_advertising', 'file_name'] 




In [ ]:
drop = ["SE_CATEGORY","SE_VALUE","GEO_CITY", "GEO_REGION", "REFR_MEDIUM",'privacy_functional',
       'privacy_analytics', 'privacy_target_advertising',
       'privacy_personalisation', 'privacy_non-personalised_ads',
       'privacy_marketing', 'privacy_social_media', 'privacy_geo_location',
       'privacy_advertising', 'file_name']

### Load article data

In [2]:
DATA_FOLDER_ARTICLES = "reduced"
ddf_article = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER_ARTICLES}/reduced_articles_df_article.csv', dtype={'sub_section': 'object', 'url': 'object'})
ddf_article = ddf_article.set_index('article_id')
ddf_article.index = ddf_article.index.astype('str')
ddf_article.head()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/Alessandra/Documents/work/DigitalHumanitiesLab/dpg/ClusterProfiles/code/data_dpg_testdata/reduced/reduced_articles_df_article.csv'

In [8]:
ddf_topics = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER_ARTICLES}/reduced_articles_df_fixed_set_topics.csv', dtype={"article_id":"str"})
topic_columns = ddf_topics.columns[ddf_topics.columns.str.startswith("topic_")].values
ddf_topics = ddf_topics.reset_index().set_index('index')
ddf_topics = ddf_topics.assign(article_topic_entropy = dd.from_array(entropy(ddf_topics[topic_columns], axis=1)))
ddf_topics = ddf_topics.assign(argmax_topic = dd.from_array(ddf_topics[topic_columns].idxmax(axis=1).compute().values))
ddf_topics = ddf_topics.set_index('article_id')
ddf_topics.head()

,topic_voetbal,topic_optredens_en_voorstellingen,topic_politiek,topic_technologie_en_handel,topic_relatie_en_opvoeden,topic_bestuur_en_organisatie,topic_human_interest,topic_verkeersongevallen,topic_nominatie_en_prijzen,topic_veiligheid,...,topic_overheidsbeleid,topic_vakbond_club_en_vereniging,topic_religie,topic_bouwen_en_wonen,topic_recht_en_justitie,topic_samenleving,topic_woon_en_leefomgeving,file_name,article_topic_entropy,argmax_topic
article_id,,,,,,,,,,,,,,,,,,,,,
1013291,0.105175,0.028085,0.001613,0.025450,0.024846,0.001953,0.036237,0.063389,0.041297,0.001265,...,0.024950,0.049110,0.001384,0.001251,0.001558,0.001142,0.054470,2,2.660889,topic_duurzaamheid
1149251,0.096021,0.001019,0.000872,0.021542,0.002109,0.001056,0.011584,0.148419,0.000740,0.000684,...,0.001159,0.014120,0.000748,0.000677,0.000842,0.000617,0.039665,3,2.116882,topic_duurzaamheid
1401797,0.002146,0.001896,0.001623,0.002359,0.004391,0.001965,0.002967,0.001080,0.022382,0.001273,...,0.002157,0.019468,0.001393,0.001259,0.001567,0.001149,0.003774,3,2.471787,topic_kunst_en_cultuur
1420717,0.001877,0.032115,0.001420,0.044774,0.003638,0.063106,0.002595,0.068904,0.001204,0.001114,...,0.001887,0.155525,0.001218,0.001101,0.001371,0.001005,0.022787,3,2.527454,topic_vakbond_club_en_vereniging
1505884,0.002216,0.001958,0.023487,0.002437,0.004154,0.002029,0.003056,0.093302,0.057185,0.016235,...,0.002224,0.020908,0.001437,0.065164,0.001618,0.001186,0.051443,3,2.486128,topic_duurzaamheid


### Load userneeds data

In [9]:
ddf_userneeds = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER_ARTICLES}/reduced_articles_df_fixed_set_userneeds.csv', dtype={'article_id':'str'})
columns_userneeds = ddf_userneeds.columns[ddf_userneeds.columns.str.startswith("userneed_")].values
ddf_userneeds = ddf_userneeds.assign(argmax_userneeds = ddf_userneeds[columns_userneeds].idxmax(axis=1))
ddf_userneeds = ddf_userneeds.set_index('article_id')
ddf_userneeds.head()

,userneed_hou_me_op_de_hoogte,userneed_geef_me_context,userneed_vermaak_me,userneed_raak_me_verbind_me,userneed_help_me,file_name,argmax_userneeds
article_id,,,,,,,
1013291,0.132751,0.059254,0.704135,0.089935,0.013925,2,userneed_vermaak_me
1149251,0.159672,0.108233,0.444364,0.015201,0.272529,3,userneed_vermaak_me
1401797,0.071099,0.054092,0.790514,0.059013,0.025282,3,userneed_vermaak_me
1420717,0.393687,0.159692,0.321118,0.030522,0.094981,3,userneed_hou_me_op_de_hoogte
1505884,0.149060,0.400487,0.137633,0.304317,0.008503,3,userneed_geef_me_context


### Load sensitivity data

In [10]:
ddf_sensitivity = dd.read_csv(f'data_dpg_testdata/{DATA_FOLDER_ARTICLES}/reduced_articles_df_fixed_set_sensitive.csv', dtype={'article_id':'str'})
columns_sensitivity = list(ddf_sensitivity.columns[ddf_sensitivity.columns.str.startswith("sensitive_")].values)
ddf_sensitivity = ddf_sensitivity.assign(argmax_sensitivity = ddf_sensitivity[columns_sensitivity].idxmax(axis=1))
ddf_sensitivity = ddf_sensitivity.assign(is_sensitive = ddf_sensitivity[columns_sensitivity].sum(axis=1))
ddf_sensitivity = ddf_sensitivity.dropna(subset="sensitive_topic_terrorism")
ddf_sensitivity = ddf_sensitivity.assign(article_sensitivity_entropy = ddf_sensitivity[columns_sensitivity].apply(lambda x: entropy(x) if x.sum()!=0 else 0, axis=1, meta='float64'))
ddf_sensitivity = ddf_sensitivity.set_index('article_id')
# ddf_sensitivity = ddf_sensitivity.persist()
ddf_sensitivity.head()

/home/bestname/anaconda3/envs/dpg/lib/python3.10/site-packages/dask/dataframe/core.py:6751: FutureWarning: Meta is not valid, `map_partitions` and `map_overlap` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


,sensitive_topic_adult,sensitive_topic_arms,sensitive_topic_crime,sensitive_topic_death,sensitive_topic_drugs,sensitive_topic_hate_speech,sensitive_topic_obscenity,sensitive_topic_piracy,sensitive_topic_sensitive,sensitive_topic_social_issue,sensitive_topic_terrorism,sensitive_topic_sensitive_and_social_issue,sensitive_topic_spam_and_harmful_content,file_name,argmax_sensitivity,is_sensitive,article_sensitivity_entropy
article_id,,,,,,,,,,,,,,,,,
1013291,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.341225,0.0,0.341225,0.317549,2,sensitive_topic_social_issue,1.0,1.098047
1149251,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,3,sensitive_topic_adult,0.0,0.000000
1401797,0.0,0.0,0.0,0.132023,0.0,0.0,0.0,0.0,0.0,0.433988,0.0,0.433988,0.000000,3,sensitive_topic_social_issue,1.0,0.991850
1420717,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.500000,0.000000,3,sensitive_topic_social_issue,1.0,0.693147
1505884,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,3,sensitive_topic_adult,0.0,0.000000


# Load samples

In [2]:
# Article data

ddf_articles = dd.read_csv("../data/sample_articles.csv")
#ddf_articles.head(2)

# Select relevant columns
ddf_arts_short = ddf_articles[["title", "num_words", "categories_generated", "keywords_curated"]]
ddf_arts_short.head()

,title,num_words,categories_generated,keywords_curated
0,Peugeot 307 in alle opzichten verbeterd,351,Onderweg,NaN
1,Hier is hij dan: de nieuwe Volvo V70!,0,Onderweg,NaN
2,De Peugeot 207 SW!,170,Onderweg,NaN
3,Meer details nieuwe Opel Agila,361,Onderweg,NaN
4,Oranje meest populair voor Dodge Charger,178,Onderweg,NaN


In [3]:
ddf_views = dd.read_csv("../data/interaction_data.csv", assume_missing = True, sep = ";")
ddf_views.head()

,ARTICLE_ID,APP_ID,DERIVED_TSTAMP,GEO_COUNTRY,GEO_ZIPCODE,REFR_URLHOST,QUASI_USER_ID,IS_LOGGED_IN,dt_hour,dt_weekday,dt_dayofmonth,dt_month,SE_LABEL,SE_ACTION
0,6221051.0,nu.web,01/09/2022 20:15,NL,7071.0,3.0,6cfbc147ff75fafd27f0ff0c507ef326,1.0,18.0,3.0,1.0,9.0,new_reaction,post_reaction
1,6221091.0,nu.ios,01/09/2022 07:44,NL,1031.0,5.0,735d4a9f97ba8a359098338320aa8164,0.0,5.0,3.0,1.0,9.0,new_reaction,post_reaction
2,6221108.0,nu.ios,31/08/2022 20:11,NL,3033.0,5.0,bbf534b9b2be63b5dc581c8ff9993e6e,0.0,18.0,2.0,31.0,8.0,new_reaction,post_reaction
3,6221144.0,nu.ios,01/09/2022 07:25,NL,1186.0,5.0,a2f6212712e17df13778e86841261ade,0.0,5.0,3.0,1.0,9.0,new_reaction,post_reaction
4,6221157.0,nu.android,01/09/2022 08:26,NL,3071.0,5.0,da0952157ee6cb342db35338d2288309,0.0,6.0,3.0,1.0,9.0,new_reaction,post_reaction


In [4]:
# Interaction data
#ddf_views = dd.read_csv("../data/views_sample.csv", assume_missing = True)
ddf_views = dd.read_csv("../data/interaction_data.csv", assume_missing = True, sep = ";")
ddf_views['ARTICLE_ID'] = ddf_views['ARTICLE_ID'].astype(int)
ddf_views = ddf_views.drop("GEO_ZIPCODE", axis = 1)
#ddf_views = ddf_views.drop("article_id", axis = 1)


In [5]:
ddf_views.head()

,ARTICLE_ID,APP_ID,DERIVED_TSTAMP,GEO_COUNTRY,REFR_URLHOST,QUASI_USER_ID,IS_LOGGED_IN,dt_hour,dt_weekday,dt_dayofmonth,dt_month,SE_LABEL,SE_ACTION
0,6221051,nu.web,01/09/2022 20:15,NL,3.0,6cfbc147ff75fafd27f0ff0c507ef326,1.0,18.0,3.0,1.0,9.0,new_reaction,post_reaction
1,6221091,nu.ios,01/09/2022 07:44,NL,5.0,735d4a9f97ba8a359098338320aa8164,0.0,5.0,3.0,1.0,9.0,new_reaction,post_reaction
2,6221108,nu.ios,31/08/2022 20:11,NL,5.0,bbf534b9b2be63b5dc581c8ff9993e6e,0.0,18.0,2.0,31.0,8.0,new_reaction,post_reaction
3,6221144,nu.ios,01/09/2022 07:25,NL,5.0,a2f6212712e17df13778e86841261ade,0.0,5.0,3.0,1.0,9.0,new_reaction,post_reaction
4,6221157,nu.android,01/09/2022 08:26,NL,5.0,da0952157ee6cb342db35338d2288309,0.0,6.0,3.0,1.0,9.0,new_reaction,post_reaction


In [6]:
# Userneeds data
ddf_needs = dd.read_csv("../Data/userneeds_sample.csv", sep = ";")
ddf_needs = ddf_needs.drop("file_name", axis = 1)


ddf_needs.head()

,ARTICLE_ID,QUASI_USER_ID,userneed_geef_me_context,userneed_help_me,userneed_hou_me_op_de_hoogte,userneed_raak_me_verbind_me,userneed_vermaak_me
0,6221650,5ca2ac94fb5918645a2b3eb386d15824,0.119574,0.044533,0.123084,0.084269,0.628540
1,6221652,0319480f035fcf216149dd68cfa7fb10,0.017739,0.002837,0.009671,0.961938,0.007814
2,6221699,08f41051f4a4c7b2f3fa728176536b04,0.762814,0.006565,0.121169,0.034335,0.075117
3,6221703,4e66c57b61aa18b7e45e8fa743ef985c,0.615400,0.023028,0.100051,0.056694,0.204827
4,6221742,13b3a9adbd5244105a270d4d56a46b3a,0.027911,0.003596,0.031106,0.922088,0.015299


In [7]:
# Sensitive data

# specify column dtypes as a dictionary


ddf_sensitive = dd.read_csv("../data/sensitive_sample.csv", sep = ";")
ddf_sensitive = ddf_sensitive.rename(columns={"article_id": "ARTICLE_ID"})


ddf_sensitive.head()

,ARTICLE_ID,file_name,sensitive_topic_adult,sensitive_topic_arms,sensitive_topic_crime,sensitive_topic_death,sensitive_topic_drugs,sensitive_topic_hate_speech,sensitive_topic_obscenity,sensitive_topic_piracy,sensitive_topic_sensitive,sensitive_topic_sensitive_and_social_issue,sensitive_topic_social_issue,sensitive_topic_spam_and_harmful_content,sensitive_topic_terrorism
0,6221650,0,0.000000,0.000000,0.000000,0.000000,0,0.0,1.000000,0.00000,0.000000,0.000000,0.00000,0.0,0.000000
1,6221652,0,0.078705,0.030634,0.177198,0.261447,0,0.0,0.000000,0.01714,0.016039,0.147409,0.13137,0.0,0.140057
2,6221699,0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.363211,0.00000,0.318395,0.318395,0.00000,0.0,0.000000
3,6221703,0,0.000000,0.501581,0.000000,0.000000,0,0.0,0.498419,0.00000,0.000000,0.000000,0.00000,0.0,0.000000
4,6221742,0,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.0,0.000000


In [8]:
# match article_id columns
article_ids = ddf_needs["ARTICLE_ID"]
ddf_arts_short["ARTICLE_ID"] = article_ids
ddf_arts_short.head()


,title,num_words,categories_generated,keywords_curated,ARTICLE_ID
0,Peugeot 307 in alle opzichten verbeterd,351,Onderweg,NaN,6221650
1,Hier is hij dan: de nieuwe Volvo V70!,0,Onderweg,NaN,6221652
2,De Peugeot 207 SW!,170,Onderweg,NaN,6221699
3,Meer details nieuwe Opel Agila,361,Onderweg,NaN,6221703
4,Oranje meest populair voor Dodge Charger,178,Onderweg,NaN,6221742


In [9]:
ddf_needs = ddf_needs.drop("ARTICLE_ID", axis = 1)

### Timing

In [5]:
%load_ext autotime

time: 1.15 ms (started: 2023-03-24 11:27:13 +01:00)


# Preprocessing

TO DO: 
- merge dataframes (interactions, views, articles) by matching them on column variables they have in common
- map strings to numerical values
- remove unneccesary columns 


### Merge dataframes 

In [6]:
# Merge dataframes 

# Merge the two dataframes on the 'article_id' column
merged_df = dd.merge(ddf_views, ddf_arts_short, on='ARTICLE_ID')
# compute the result
result1 = merged_df.compute()

# Repeat to include the userneeds dataframe
merged_df2 = dd.merge(result1, ddf_needs, on="QUASI_USER_ID")
result2 = merged_df2.compute()

# Repeat to include the sensitive dataframe
merged_df3 = dd.merge(result2, ddf_sensitive, on="ARTICLE_ID")
df = merged_df3.compute()

# From now on, continue to work with this dataframe, it contains all relevant columns
df

NameError: name 'ddf_views' is not defined

time: 33.9 ms (started: 2023-03-24 11:27:18 +01:00)


In [11]:
# Add keywords 
keywords = ["Max Verstappen|Formule 1", "Geld", "Coronavirus", "Klimaat", "Oorlog in Oekraïne|Buitenland",
            "Media|Media en Cultuur", "Wielrennen|Tour de France", "woordle", "Formule 1|Max Verstappen",
            "Economie|Oorlog in Oekraïne"] 
df["keywords_curated"] = keywords

In [12]:
# Keyword features 

def split_strings(keyword_list): 
    """
    :input: list with strings
    :output: nested list with strings 
    
    
    This function splits the contents of a list on the "|" delimiter
    and returns the splitted strings
    """

    split_cats = []

    for li in keyword_list:
        split_string = li.split("|")
        split_cats.append(split_string)

    return split_cats


In [13]:
## Split the categories_generated variable into strings so they can be processed separately
categories = df["keywords_curated"]
split_categories = split_strings(categories)

## Replace the old strings with the splitted ones
df["keywords_numerical"] = split_categories
df["keywords_embeddings"] = split_categories

## Do the same for the curated keywords 
#keywords = df["keywords_curated"]
#print(keywords)
#split_keywords = split_strings(keywords)
#df["keywords_curated"] = split_keywords

In [14]:
df["keywords_numerical"]

0         [Max Verstappen, Formule 1]
1                              [Geld]
2                       [Coronavirus]
3                           [Klimaat]
4    [Oorlog in Oekraïne, Buitenland]
5           [Media, Media en Cultuur]
6        [Wielrennen, Tour de France]
7                           [woordle]
8         [Formule 1, Max Verstappen]
9      [Economie, Oorlog in Oekraïne]
Name: keywords_numerical, dtype: object

### Make mappings 

In [15]:
def replace_items(lst, mapping):
    """
    This function replaces items in a list that possibly contains nested items
    """
    return [replace_items(sublst, mapping) if isinstance(sublst, list) else mapping.get(sublst, sublst) for sublst in lst]


def make_mapping(df, variable):
    """
    You can use this function to map variables in a dataframe to a number. Just feed the 
    dataframe and column name into this function. 

    :input df:        Dataframe 
    :input variable:  String, name of the column you want to map to numbers
    :output df:       Dataframe
    
    """
    
    column_list = df[variable]
    
    # Get the unique list values while preserving the input order 
    
    
    ### Use this loop for columns with nested lists
    if variable == 'keywords_numerical':
        # Explode the list of strings into separate rows
        df_exploded = df.explode(variable)
        unique_values = df_exploded[variable].unique()
        
    ### Use this loop for values that a re strings 
    else: 
        unique_values = list(OrderedDict.fromkeys(column_list))
    
    # Loop over the unique values and add the mappings to a lookup dictionary
    mapping = {}
    for i, s in enumerate(unique_values, 1):
        mapping[s] = i
        

    if variable == "keywords_numerical": 
        new_list = replace_items(column_list, mapping)
        #new_list = [mapping.get(item, item) for item in column_list]
        df[variable] = new_list
    
    else: 
        # Replace values using the lookup dictionary
        df = df.replace({variable: mapping})
    
    return df

    

In [16]:
# Apply mappings 

df = make_mapping(df, "GEO_COUNTRY")
df = make_mapping(df, "APP_ID")
#df = make_mapping(df, "REFR_MEDIUM")
df = make_mapping(df, "QUASI_USER_ID")
df = make_mapping(df, "SE_LABEL")
df = make_mapping(df, "SE_ACTION") 
df = make_mapping(df, "keywords_numerical")



df.head()

,ARTICLE_ID,APP_ID,DERIVED_TSTAMP,GEO_COUNTRY,REFR_URLHOST,QUASI_USER_ID,IS_LOGGED_IN,dt_hour,dt_weekday,dt_dayofmonth,...,sensitive_topic_hate_speech,sensitive_topic_obscenity,sensitive_topic_piracy,sensitive_topic_sensitive,sensitive_topic_sensitive_and_social_issue,sensitive_topic_social_issue,sensitive_topic_spam_and_harmful_content,sensitive_topic_terrorism,keywords_numerical,keywords_embeddings
0,6221650,1,03/09/2022 01:53,1,5.0,1,0.0,23.0,4.0,2.0,...,0.0,1.000000,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,"[1, 2]","[Max Verstappen, Formule 1]"
1,6221652,2,03/09/2022 08:49,2,5.0,2,0.0,6.0,5.0,3.0,...,0.0,0.000000,0.01714,0.016039,0.147409,0.13137,0.0,0.140057,[3],[Geld]
2,6221699,3,06/09/2022 16:44,2,1.0,3,1.0,14.0,1.0,6.0,...,0.0,0.363211,0.00000,0.318395,0.318395,0.00000,0.0,0.000000,[4],[Coronavirus]
3,6221703,3,03/09/2022 19:23,2,3.0,4,0.0,17.0,5.0,3.0,...,0.0,0.498419,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,[5],[Klimaat]
4,6221742,3,03/09/2022 16:48,2,1.0,5,1.0,14.0,5.0,3.0,...,0.0,0.000000,0.00000,0.000000,0.000000,0.00000,0.0,0.000000,"[6, 7]","[Oorlog in Oekraïne, Buitenland]"


In [17]:
df["keywords_embeddings"]

0         [Max Verstappen, Formule 1]
1                              [Geld]
2                       [Coronavirus]
3                           [Klimaat]
4    [Oorlog in Oekraïne, Buitenland]
5           [Media, Media en Cultuur]
6        [Wielrennen, Tour de France]
7                           [woordle]
8         [Formule 1, Max Verstappen]
9      [Economie, Oorlog in Oekraïne]
Name: keywords_embeddings, dtype: object

In [18]:
df.columns

Index(['ARTICLE_ID', 'APP_ID', 'DERIVED_TSTAMP', 'GEO_COUNTRY', 'REFR_URLHOST',
       'QUASI_USER_ID', 'IS_LOGGED_IN', 'dt_hour', 'dt_weekday',
       'dt_dayofmonth', 'dt_month', 'SE_LABEL', 'SE_ACTION', 'title',
       'num_words', 'categories_generated', 'keywords_curated',
       'userneed_geef_me_context', 'userneed_help_me',
       'userneed_hou_me_op_de_hoogte', 'userneed_raak_me_verbind_me',
       'userneed_vermaak_me', 'file_name', 'sensitive_topic_adult',
       'sensitive_topic_arms', 'sensitive_topic_crime',
       'sensitive_topic_death', 'sensitive_topic_drugs',
       'sensitive_topic_hate_speech', 'sensitive_topic_obscenity',
       'sensitive_topic_piracy', 'sensitive_topic_sensitive',
       'sensitive_topic_sensitive_and_social_issue',
       'sensitive_topic_social_issue',
       'sensitive_topic_spam_and_harmful_content', 'sensitive_topic_terrorism',
       'keywords_numerical', 'keywords_embeddings'],
      dtype='object')

# Sampling from four datasets

In [ ]:
df1_sample = ddf_articles.sample(n=500)
df2_sample = ddf_views.sample(n=500)
df3_sample = ddf_userneeds.sample(n=500)
df4_sample = ddf_sensitivity.sample(n=500)

merged_df = pd.concat([df1_sample, df2_sample, df3_sample, df4_sample])

#merged_df.to_csv("merged_dataset.csv", index=False)

# Build features

In [18]:
# Load embedding model 

#model_name = "GroNLP/bert-base-dutch-cased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModel.from_pretrained(model_name)


In [19]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("DTAI-KULeuven/robbertje-1-gb-non-shuffled")
model = AutoModelForMaskedLM.from_pretrained("DTAI-KULeuven/robbertje-1-gb-non-shuffled")

Downloading:   0%|          | 0.00/644 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/715k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/374k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/283M [00:00<?, ?B/s]

In [39]:
def get_embedding(input_string):
    """
    :input: string 
    :output: embedding of 384 dimensions
    
    """
    
    #embedding = model_sentence.encode(string)    
    #list_embedding = embedding.tolist()

    # Tokenize the input string
    input_encoding = tokenizer(input_string, return_tensors='pt')
    
    # Get the outputs of the model
    outputs = model(**input_encoding)
    hidden_states = outputs.hidden_states
    
    # Get the word embeddings by taking the mean of the last hidden state over the token dimensions
    last_hidden_state = hidden_states[-1]
    word_embeddings = torch.mean(last_hidden_state, dim=1).squeeze(0)
    
    list_embedding = word_embeddings.tolist()
    
    return list_embedding
    
    

In [21]:
def sensitive_mapping(df): 
    """
    :input: dataframe 
    
    """
    adult = df['sensitive_topic_adult'] 
    arms = df['sensitive_topic_arms'] 
    crime = df['sensitive_topic_crime'] 
    death = df['sensitive_topic_death'] 
    drugs = df['sensitive_topic_drugs'] 
    hatespeech = df['sensitive_topic_hate_speech'] 
    obscenity = df['sensitive_topic_obscenity'] 
    piracy = df['sensitive_topic_piracy'] 
    sensitive = df['sensitive_topic_sensitive']
    sens_social = df['sensitive_topic_sensitive_and_social_issue'] 
    social = df['sensitive_topic_social_issue']
    spam = df['sensitive_topic_spam_and_harmful_content'] 
    terrorism = df['sensitive_topic_terrorism'] 
    
    vector = [adult, arms, crime, death, drugs, hatespeech, obscenity, 
              piracy, sensitive, sens_social, social, spam, terrorism]
    
    return list(vector)

In [22]:
# User needs mapping
def userneeds_mapping(df):
    '''
    Input: 

    Output: Vector containing the probabilities of 5 userneeds
    '''
    # Define a function that returns a vector of values from multiple columns in a DataFrame
    
    context = df["userneed_geef_me_context"]
    helpme = df['userneed_help_me']
    hoogte = df['userneed_hou_me_op_de_hoogte'] 
    raakme = df['userneed_raak_me_verbind_me']
    vermaakme = df['userneed_vermaak_me']
    
    vector = [context, helpme, hoogte, raakme, vermaakme]
    

    return list(vector)

In [23]:
def get_keyword_vector(keywords): 
    """
    This function makes sure the length of the keyword features is the same length 
    regardless of the number of categories present in the row.
    
    :input: list of 1 or more keyword
    :output: 2 ints representing each keyword
    
    """
    
    # If the list contains two or more items
    if len(keywords) >= 2:
        key1 = keywords[0]
        key2 = keywords[1]
    
    
    # Otherwise, add a 0 to indicate the missing value 
    else: 
        key1 = keywords[0]
        key2 = 0
    
    return key1, key2
        
    

In [54]:
def make_feature_vector(df):
    """
    input: row of a dataframe 
    
    output: a list containing 1 feature vector
    
    In this function, the feature vectors are made. In some cases, the values are taken 
    directly from the input row (e.g. the straightforward features), sometimes 
    other functions are called in to transform the features (e.g. embeddings, sensitive topics)
    
    """
    
    #### STRAIGHTFORWARD FEATURES #### 
    ## Select the columns that can be directly converted into features 
    
    article_id = int(df["ARTICLE_ID"])
    user_id = int(df["QUASI_USER_ID"])
    app_id = int(df["APP_ID"])
    hour = int(df["dt_hour"])
    weekday = int(df["dt_weekday"])
    
    login = int(df["IS_LOGGED_IN"])
    num_words = int(df["num_words"]) # In the article data, a lot of articles have 0 in this column. That could skew the results
    
    interaction_labels = int(df["SE_LABEL"])
    interaction_actions = int(df["SE_ACTION"])
    
    #### EMBEDDING FEATURES ####
    #title = df["title"]
    #title_embedding = get_embedding(title)

    
    #### USERNEEDES ####
    #userneeds = userneeds_mapping(df["userneed_geef_me_context"]), userneeds_mapping(df["userneed_help_me"]), userneeds_mapping(df["userneed_hou_me_op_de_hoogte"]), userneeds_mapping(df['userneed_raak_me_verbind_me']), userneeds_mapping(df['userneed_vermaak_me'])
    
    
    #### SENSITIVE TOPICS + USERNEEDS  #### 
    ## note: these variables contain a list 
    sensitive_topics = sensitive_mapping(df)
    userneeds = userneeds_mapping(df)

    
    #### GENERATED CATEGORIES ####
    keywords_numerical = df["keywords_numerical"]
    keywords_embeddings = df["keywords_embeddings"]
    # Apply this function to ensure the same length
    key_num1, key_num2 = get_keyword_vector(keywords_numerical)
    key_emb1, key_emb2 = get_keyword_vector(keywords_embeddings)
    
    # Get embeddings features 
    emb1 = get_embedding(str(key_emb1))
    emb2 = get_embedding(str(key_emb2))
    
    ## Make vector     
    vector = [article_id, user_id, app_id, hour, weekday, login, num_words, 
              interaction_labels, interaction_actions, key_num1, key_num2]
    
    
    ## Iterate over the sensitive topic vector 
    ## as the final vector should only contain elements, not lists
    for sensitive_topic in sensitive_topics: 
        vector.append(float(sensitive_topic))
        
    ## Do the same for the userneeds: 
    for need in userneeds: 
        vector.append(float(need))
    
    
    ## And for the embeddings: 
    for embedding in emb1: 
        vector.append(embedding)
    for embedding in emb2: 
        vector.append(embedding)
    
    
    
    return vector
    

# Apply features 

In [55]:
# The .apply function performs the make_feature_vector function to all rows in the dataframe
features = df.apply(make_feature_vector, axis = 1)
features = features.tolist()

In [56]:
# Inspect features 
print(type(features))

for f in features: 
    print(len(f))
    print(f)
    print()
    print()

<class 'list'>
1565
[6221650, 1, 1, 23, 4, 0, 351, 1, 1, 1, 2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.11957381, 0.044533264, 0.12308365, 0.08426891, 0.6285405, -0.7714215517044067, 0.1212751716375351, -0.8421460390090942, 0.06853185594081879, -0.4287978708744049, -0.18140752613544464, -0.1023041307926178, -0.4480718970298767, 0.40302741527557373, -0.13763882219791412, -0.2503593862056732, 0.03541667014360428, 0.47454553842544556, 0.05343073233962059, -0.8490358591079712, 0.06981077790260315, 0.3528658151626587, 0.12499546259641647, 0.03291439265012741, 0.28164243698120117, 0.8720963001251221, 0.1558842957019806, -0.09963485598564148, -0.6546840667724609, 0.08198481053113937, 0.47653743624687195, -0.16033083200454712, 0.14496609568595886, -0.025973469018936157, 0.14970244467258453, -0.3542018532752991, 0.21204760670661926, -0.5595376491546631, 0.1673712134361267, -0.14500269293785095, -0.3083185851573944, -0.9657764434814453, 0.08453913778066635, 35.443504333

# K-Means Clustering

In [48]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score 
from sklearn.preprocessing import StandardScaler

In [59]:
# Initiate clustering model settings
kmeans = KMeans(n_clusters = 5, random_state = 0)

# Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Fit the clustering model and generate labels 
kmeans.fit(scaled_features)
labels = kmeans.predict(scaled_features)

print(labels)

[1 2 2 2 0 3 0 2 4 0]


In [60]:
# Silhouette score 
sil = silhouette_score(scaled_features, kmeans.labels_, metric='euclidean')
print(sil)

0.1406323658070721
